In [28]:
# import libraries
import cv2
import os
from PIL import Image
import datetime
import numpy as np
import datetime
import numpy as np
import math
import pandas as pd

In [29]:
def dni_rect_from_ce(x, y, cv2_v, img, areas_dict, lines_dict):
    w = img.shape[1]
    h = img.shape[0]

    color = (0, 0, 255)
    thicknes = 2

    fields = []
    res_dict = {}

    for key in areas_dict.keys():
        tras_vector = np.array([areas_dict[key]['tras_x'], areas_dict[key]['tras_y']])
        point = np.array([x, y]) + tras_vector
        point_2 = point + np.array([areas_dict[key]['width'], areas_dict[key]['height']])

        point = np.where(point<=0,0,point)
        point_2[0] = w if point_2[0] >= w else point_2[0]
        point_2[1] = h if point_2[1] >= h else point_2[1]

        cv2_v.rectangle(img,(point[0],point[1]),(point_2[0],point_2[1]),color,thicknes)
        res_dict[key] = (point, point_2)
        # print('{} {}'.format(point,point_2))

    for key in lines_dict.keys():
        tras_vector = np.array([lines_dict[key]['tras_x'], lines_dict[key]['tras_y']])
        point = np.array([x, y]) + tras_vector
        point_2 = point + np.array([lines_dict[key]['width'], lines_dict[key]['height']])

        point = np.where(point<=0,0,point)
        point_2[0] = w if point_2[0]>= w else point_2[0]
        point_2[1] = h if point_2[1]>= h else point_2[1]

        cv2_v.rectangle(img,(point[0],point[1]),(point_2[0],point_2[1]),color,thicknes)
        res_dict[key] = (point, point_2)

    return res_dict['dni'][0], res_dict['dni'][1], res_dict['tel'][0], res_dict['tel'][1]
    # return res_dict


In [44]:
dni_search_area = {
    'x_min':0,
    'x_max':695,
    'y_min':105,
    'y_max':195
}
traslation_dict = {
    'nomb_ape1':{'tras_x':227,'tras_y':-145,'width':400,'height':60},
    'nomb_ape2':{'tras_x':-38,'tras_y':-90,'width':678,'height':62},
    'dni':{'tras_x':24,'tras_y':-34,'width':617,'height':50},
    'tel':{'tras_x':84,'tras_y':163,'width':559,'height':57},
    'dir1':{'tras_x':97,'tras_y':14,'width':545,'height':54},
    'dir2':{'tras_x':-42,'tras_y':64,'width':684,'height':57},
    'distrito':{'tras_x':69,'tras_y':117,'width':573,'height':52},
    'mail':{'tras_x':59,'tras_y':214,'width':585,'height':80}
}
lines_dict = {

}

In [34]:
# SIFT initialization
detector = cv2.xfeatures2d.SIFT_create()
FLANN_INDEX_KDITREE = 0
flannParam = dict(algorithm=FLANN_INDEX_KDITREE, tree=5)
flann = cv2.FlannBasedMatcher(flannParam, {})
trainImg = cv2.imread("sift_img/dni5.jpg", 0)
trainKP, trainDesc = detector.detectAndCompute(trainImg, None)
MIN_MATCH_COUNT = 20

In [32]:
# 1 Getting files in multiple paths
# ruta = 'D:\\GyS\\proyectos\\cuponesWong\\proceso_definitivo\\escaneos_concursos'
# ruta = 'C:\\git\\cuponesWong\\CuponesWong\\data\\escaneos\\camioneta_sonada_2018'
ruta = 'D:\\GyS\\proyectos\\cuponesWong\\eWong\\VivaItalia2019'
rutas_personas  = [arch.name for arch in os.scandir(ruta) if arch.is_dir()]
total = 0

files_lista = []

for persona in rutas_personas:
    bolsas  = [arch.name for arch in os.scandir(ruta + "\\" + persona) if arch.is_dir()]
    for bolsa in bolsas:
        files = [arch.name for arch in os.scandir(ruta + "\\" + persona + "\\" + bolsa) if arch.is_file()]
        for file in files:
            files_lista.append(ruta + "\\" + persona + "\\" + bolsa + "\\" + file)
print("Terminó. {}".format(len(files_lista)))

Terminó. 122096


In [6]:
# 1 Getting files from inputh
ruta = 'C:\\git\\cuponesWong\\CuponesWong\\data\\escaneos\\viaje_italia_2019'
files = [arch.name for arch in os.scandir(ruta) if arch.is_file()]
files_lista = []

for i in files:
    files_lista.append(ruta+'\\'+i)

print("Terminó. {}".format(len(files_lista)))

Terminó. 7


In [42]:
# Extracción del area de forma estática
y_min = dni_search_area['y_min']
y_max = dni_search_area['y_max']
x_min = dni_search_area['x_min']
x_max = dni_search_area['x_max']

INPUT = True
falses = 0

# array's
h_lista = []
w_lista = []
p_lista = []
c_x_lista = []
c_y_lista = []
file_lista = []
paths_lista = []

print("Inicio {}".format(datetime.datetime.now()))
# for filepath in files_lista[1730:1790]:
for filepath in files_lista[::8]:
    # Initializing a boolean variable with False
    dni_founded = False

    if INPUT:
        print('Reading {}'.format(filepath))
        # Preprocessing section
        image = cv2.imread(filepath)
    else:
        print('Reading {}'.format(filepath[0]))
        # Preprocessing section
        image = cv2.imread(filepath[0])

    # print('Reading {}'.format(filepath))
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # print('{}'.format(image.shape))
    y_max = image.shape[0] if y_max > image.shape[0] else y_max
    x_max = image.shape[1] if x_max > image.shape[1] else x_max

    sift_roi = gray[y_min:y_max, x_min:x_max]
    sift_roi = cv2.adaptiveThreshold(sift_roi, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 21, 11)
    
    # SIFT DETECTION
    queryKP, queryDesc = detector.detectAndCompute(sift_roi, None)
    matches = flann.knnMatch(queryDesc, trainDesc, k=2)
    detection = False

    goodMatch = []
    for m, n in matches:
        if(m.distance < 0.75 * n.distance):
            goodMatch.append(m)
    if(len(goodMatch) > MIN_MATCH_COUNT):
        tp = []
        qp = []
        for m in goodMatch:
            tp.append(trainKP[m.trainIdx].pt)
            qp.append(queryKP[m.queryIdx].pt)
        tp, qp = np.float32((tp, qp))
        H, status = cv2.findHomography(tp, qp, cv2.RANSAC, 3.0)
        h, w = trainImg.shape
        trainBorder = np.float32([[[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]])
        queryBorder = cv2.perspectiveTransform(trainBorder, H)
        cv2.polylines(sift_roi, [np.int32(queryBorder)], True, (0, 255, 0), 2)
        dni_founded = True
        
    if dni_founded:
        # -------Comparacion de Hmax y Wmax-----
        # restringinedo valores a 0
        queryBorder = np.where(queryBorder < 0, 0, queryBorder)

        # Coordenadas "query"
        x0 = queryBorder[[[0],0]][[0],0]
        y0 = queryBorder[[[0],0]][[0],1]

        x1 = queryBorder[[[0],1]][[0],0]
        y1 = queryBorder[[[0],1]][[0],1]

        x2 = queryBorder[[[0],2]][[0],0]
        y2 = queryBorder[[[0],2]][[0],1]

        x3 = queryBorder[[[0],3]][[0],0]
        y3 = queryBorder[[[0],3]][[0],1]

        arr_x = np.array([x0, x1, x2, x3])
        x_maximo = arr_x.max()
        x_minimo = arr_x.min()
        c_x = int(arr_x.sum() / arr_x.size)

        arr_y = np.array([y0, y1, y2, y3])
        y_maximo = arr_y.max()
        y_minimo = arr_y.min()
        c_y = int(arr_y.sum() / arr_y.size)

        h_lista.append(y_maximo - y_minimo)
        w_lista.append(x_maximo - x_minimo)
        p_lista.append([(x0, y0), (x1, y1), (x2, y2), (x3, y3)])
        c_x_lista.append(c_x)
        c_y_lista.append(c_y)
        paths_lista.append(filepath if INPUT else filepath[0])
    else:
        falses += 1
        h_lista.append(9999)
        w_lista.append(9999)
        p_lista.append([(0,0,),(0,0,),(0,0,),(0,0,)])
        c_x_lista.append(9999)
        c_y_lista.append(9999)
        paths_lista.append(filepath if INPUT else filepath[0])

    # cv2.imshow("image", image)
    # cv2.imshow("sift_roi", sift_roi)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

# convirtiendo a nunmpy arrays
n_widths = np.array(w_lista)
n_heights = np.array(h_lista)
n_points = np.array(p_lista)
n_cxs = np.array(c_x_lista)
n_cys = np.array(c_y_lista)
n_paths = np.array(paths_lista)

data = {
    'path': n_paths,
    'width': n_widths,
    'height': n_heights,
    'cx': n_cxs,
    'cy': n_cys
}
areas_r = pd.DataFrame(data)
areas_r.to_csv('areas_result/areas.csv', index=False)
print("Termino :D {} Total de falses: {}".format(datetime.datetime.now(), falses))

Inicio 2019-11-22 00:44:11.080229
Reading D:\GyS\proyectos\cuponesWong\eWong\VivaItalia2019\AV\B32\VivaItalia2019_AV_B32_20190805131920_00001.jpg
Reading D:\GyS\proyectos\cuponesWong\eWong\VivaItalia2019\AV\B32\VivaItalia2019_AV_B32_20190805131922_00009.jpg
Reading D:\GyS\proyectos\cuponesWong\eWong\VivaItalia2019\AV\B32\VivaItalia2019_AV_B32_20190805131924_00017.jpg
Reading D:\GyS\proyectos\cuponesWong\eWong\VivaItalia2019\AV\B32\VivaItalia2019_AV_B32_20190805131926_00025.jpg
Reading D:\GyS\proyectos\cuponesWong\eWong\VivaItalia2019\AV\B32\VivaItalia2019_AV_B32_20190805131929_00033.jpg
Reading D:\GyS\proyectos\cuponesWong\eWong\VivaItalia2019\AV\B32\VivaItalia2019_AV_B32_20190805131930_00041.jpg
Reading D:\GyS\proyectos\cuponesWong\eWong\VivaItalia2019\AV\B32\VivaItalia2019_AV_B32_20190805131935_00049.jpg
Reading D:\GyS\proyectos\cuponesWong\eWong\VivaItalia2019\AV\B32\VivaItalia2019_AV_B32_20190805131938_00057.jpg
Reading D:\GyS\proyectos\cuponesWong\eWong\VivaItalia2019\AV\B32\VivaI

Reading D:\GyS\proyectos\cuponesWong\eWong\VivaItalia2019\AV\B32\VivaItalia2019_AV_B32_20190805132139_00617.jpg
Reading D:\GyS\proyectos\cuponesWong\eWong\VivaItalia2019\AV\B32\VivaItalia2019_AV_B32_20190805132139_00625.jpg
Reading D:\GyS\proyectos\cuponesWong\eWong\VivaItalia2019\AV\B32\VivaItalia2019_AV_B32_20190805132140_00633.jpg
Reading D:\GyS\proyectos\cuponesWong\eWong\VivaItalia2019\AV\B32\VivaItalia2019_AV_B32_20190805132140_00641.jpg
Reading D:\GyS\proyectos\cuponesWong\eWong\VivaItalia2019\AV\B32\VivaItalia2019_AV_B32_20190805132141_00649.jpg
Reading D:\GyS\proyectos\cuponesWong\eWong\VivaItalia2019\AV\B32\VivaItalia2019_AV_B32_20190805132141_00657.jpg
Reading D:\GyS\proyectos\cuponesWong\eWong\VivaItalia2019\AV\B32\VivaItalia2019_AV_B32_20190805132142_00665.jpg
Reading D:\GyS\proyectos\cuponesWong\eWong\VivaItalia2019\AV\B32\VivaItalia2019_AV_B32_20190805132142_00673.jpg
Reading D:\GyS\proyectos\cuponesWong\eWong\VivaItalia2019\AV\B32\VivaItalia2019_AV_B32_20190805132142_00

KeyboardInterrupt: 

In [8]:
falses

1

In [46]:
# for i in range(m_files.size):
# for i in range(areas_r.path.values.size):
for i in range(400, 450):
    # Grabbing the file
    filename = areas_r.path.values[i]
    name = filename.split('\\')
    name = name[len(name) - 1]
    name = name.split('.')[0]

    c_x = int(areas_r.cx.values[i])
    c_y = int(areas_r.cy.values[i])

    wi = areas_r.width.values[i]
    hi = areas_r.height.values[i]

    x_minimo = int(c_x - (wi / 2))
    x_maximo = int(c_x + (wi / 2))
    y_minimo = int(c_y - (hi / 2))
    y_maximo = int(c_y + (hi / 2))

    image = cv2.imread(filename)
    frame = image.copy()
    found = False

    # print('Reading {}'.format(filepath))
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # print('{}'.format(image.shape))
    y_max = image.shape[0] if y_max > image.shape[0] else y_max
    x_max = image.shape[1] if x_max > image.shape[1] else x_max

    sift_roi = gray[y_min:y_max, x_min:x_max]
    sift_roi = cv2.adaptiveThreshold(sift_roi, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 21, 11)
    
    
    if (c_x == 9999 or c_y == 9999):
        # cv2.putText(image,'{}'.format('Area no detectada'), (25, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        print('{},{}'.format(filename, 'Ancla no encontrada'))
        found = False
    else:
        found = True
        print('{} . {}'.format(i, name))
        cv2.rectangle(image,(x_minimo, y_minimo + int(y_min)), (x_maximo,y_maximo+int(y_min)), (0,0,255), 1)
        dni_rect_from_ce(c_x, c_y + y_min, cv2, image, traslation_dict, {})
    
    cv2.imshow('sift',sift_roi)
    cv2.imshow("Img", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


400 . VivaItalia2019_AV_B32_20190805170636_00198
401 . VivaItalia2019_AV_B32_20190805170636_00206
402 . VivaItalia2019_AV_B32_20190805170636_00214
403 . VivaItalia2019_AV_B32_20190805170637_00222
404 . VivaItalia2019_AV_B32_20190805170637_00230
405 . VivaItalia2019_AV_B32_20190805170637_00238
406 . VivaItalia2019_AV_B32_20190805170637_00246
407 . VivaItalia2019_AV_B32_20190805170637_00254
408 . VivaItalia2019_AV_B32_20190805170637_00262
409 . VivaItalia2019_AV_B32_20190805170638_00270
410 . VivaItalia2019_AV_B32_20190805170638_00278
411 . VivaItalia2019_AV_B32_20190805170638_00286
412 . VivaItalia2019_AV_B32_20190805170638_00294
D:\GyS\proyectos\cuponesWong\eWong\VivaItalia2019\AV\B32\VivaItalia2019_AV_B32_20190805170639_00302.jpg,Ancla no encontrada
414 . VivaItalia2019_AV_B32_20190805170639_00310
415 . VivaItalia2019_AV_B32_20190805170640_00318
416 . VivaItalia2019_AV_B32_20190805170640_00326
D:\GyS\proyectos\cuponesWong\eWong\VivaItalia2019\AV\B32\VivaItalia2019_AV_B32_2019080517064

In [9]:
image = cv2.imread('D:\\GyS\\proyectos\\cuponesWong\\eWong\\VivaItalia2019\\AV\\B32\\VivaItalia2019_AV_B32_20190805131935_00049.jpg')

In [15]:
# Extracción del area de forma estática
y_min = dni_search_area['y_min']
y_max = dni_search_area['y_max']
x_min = dni_search_area['x_min']
x_max = dni_search_area['x_max']


# print('Reading {}'.format(filepath))
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# print('{}'.format(image.shape))
y_max = image.shape[0] if y_max > image.shape[0] else y_max
x_max = image.shape[1] if x_max > image.shape[1] else x_max

sift_roi = gray[y_min:y_max, x_min:x_max]
sift_roi = cv2.adaptiveThreshold(sift_roi, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 21, 11)
    
cv2.imshow("Img", image)
cv2.imshow("sift_roi", sift_roi)

cv2.waitKey(0)
cv2.destroyAllWindows()